In [1]:
import pandas as pd
import os
import re
import time
import numpy as np
import statistics
from natsort import natsorted, ns
import random
from sklearn.preprocessing import StandardScaler 

from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from xgboost import XGBClassifier

In [2]:
def spatial_two (Node_class, Adj,n):
    F_vec=[]
    for i in range(n):
        #print("\rProcessing file {} ({}%)".format(i, 100*i//(n-1)), end='', flush=True)
        node_F=[]
        list_out=[]
        list_In=[]
        S_nbd_out=[]
        S_nbd_in=[]
        for j in range(n):
            if Adj[i][j]==1 and i!=j:
                list_out.append(Adj[j][j])
                for k in range(n):
                    if A[j][k]==1 and k!=j:
                        S_nbd_out.append(Adj[k][k])
            if Adj[j][i]==1 and i!=j:
                list_In.append(A[j][j])
                for k in range(n):
                    if Adj[k][j]==1 and k!=j:
                        S_nbd_in.append(Adj[k][k])

        for d in Node_class:
            count=0
            count_in=0
            for k in range(len(list_out)):
                if list_out[k]==d:
                    count=count+1
            node_F.append(count)
            for k in range(len(list_In)):
                if list_In[k]==d:
                    count_in=count_in+1
            node_F.append(count_in)
        for d in Node_class:
            count_S_out=0
            count_S_in=0
            for k in range(len(S_nbd_out)):
                if S_nbd_out[k]==d:
                    count_S_out=count_S_out+1
            node_F.append(count_S_out)
            for k in range(len(S_nbd_in)):
                if S_nbd_in[k]==d:
                    count_S_in=count_S_in+1
            node_F.append(count_S_in)
        F_vec.append(node_F)
    return F_vec

In [3]:
import pandas as pd
import numpy as np
fe_len=3703
# read the dataset from the link, extract and make a dataframe
import requests, zipfile, io
r = requests.get('https://linqs-data.soe.ucsc.edu/public/datasets/citeseer-doc-classification/citeseer-doc-classification.zip', stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
feature_names = ["w_{}".format(ii) for ii in range(fe_len)]
column_names = ["ID"]+ feature_names + ["subject"]
node_data = pd.read_csv("citeseer-doc-classification/citeseer.content", sep='\t', header=None, names=column_names)
node_data.head()

/var/folders/k8/fv2pzvcn5p77dgds_c16lsl80000gn/T/ipykernel_10461/227978353.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  node_data = pd.read_csv("citeseer-doc-classification/citeseer.content", sep='\t', header=None, names=column_names)


,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_3694,w_3695,w_3696,w_3697,w_3698,w_3699,w_3700,w_3701,w_3702,subject
0,100157,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agents
1,100598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,IR
2,105684,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Agents
3,11099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,DB
4,114091,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,AI


In [4]:
DataS=node_data.replace({'subject': {'Agents':0 , 'AI': 1,'DB':2,'IR':3 , 'ML': 4,'HCI':5}})
# Agents==0
#		AI==1
#		DB==2
#		IR==3
#		ML==4
#		HCI==5

DataS.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_3694,w_3695,w_3696,w_3697,w_3698,w_3699,w_3700,w_3701,w_3702,subject
0,100157,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,105684,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,114091,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
#SORT THE NODE FEATURE ACCORDING TO ID

from natsort import index_natsorted, order_by_index
Data = DataS.reindex(index=order_by_index(DataS.index, index_natsorted(DataS['ID'], reverse=False)))
#Data = Data.reset_index(drop=True)
Data.head()

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_3694,w_3695,w_3696,w_3697,w_3698,w_3699,w_3700,w_3701,w_3702,subject
29,210,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1380,845,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2509,940,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2206,1529,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2210,1625,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [6]:
Data.tail(10)

,ID,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,...,w_3694,w_3695,w_3696,w_3697,w_3698,w_3699,w_3700,w_3701,w_3702,subject
2171,zhao98empirical,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4
2172,zhao99discriminant,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4
2173,zheng98integrating,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,4
2174,zheng98stochastic,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,4
3310,zhou00implementation,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,2
1008,zhu00incorporating,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2175,zhu00segmenting,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2176,zhu98line,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1009,zini01caselp,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1010,zunino01representing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
catagories=Data['subject'].to_numpy()
n=len(catagories)
data_by_class = {cls: Data.loc[Data['subject'] == cls].drop(['subject', 'ID'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[f'w_{i}']) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[f'w_{i}'].to_numpy()).count(1) >= int(len(df[f'w_{i}'].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
IdS=list(DataS['ID'].to_numpy())
Id=list(Data['ID'].to_numpy())

In [8]:
Fec=[]
for i in range(n):
    vec=[]
    f=Data.loc[IdS.index(Id[i]), feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and basis[5][j]==1:
            count5=count5+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    vec.append(count5)
    #print(f)
    f.clear()
    Fec.append(vec)

In [9]:
SFec=[]
for i in range(n):
    Svec=[]
    f=Data.loc[IdS.index(Id[i]), feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    for j in range(1433):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and sel_basis[5][j]==1:
            count5=count5+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    Svec.append(count5)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [10]:
y =np.array(Fec)
data1=pd.DataFrame({"I_0":y[:,0]})
for i in range(5):
    data1.insert(loc=i+1, column="I_{}".format(i+1), value=y[:,i+1])
#data1.insert(loc=6,column='Class',value=Data['subject'].to_numpy())
data1.head()

,I_0,I_1,I_2,I_3,I_4,I_5
0,37,30,32,32,37,34
1,32,32,34,32,36,30
2,27,28,27,29,29,27
3,29,28,34,30,31,30
4,26,25,25,25,27,22


In [11]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(5):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4,S_5
0,1,2,0,1,2,1
1,2,3,2,2,4,1
2,2,3,2,4,5,2
3,4,5,3,5,7,4
4,2,5,2,3,5,2


In [12]:
from urllib.request import urlopen
p_data=urlopen("https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/Citeseer/citeseer.cites")
er=[]
for line in p_data:
    output = line.decode()
    er.append(output)
myls=list(er)
mylist = [line.rstrip('\n') for line in myls]
mylist.pop(0)
#print(mylist)

'100157\t100157'

In [13]:
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(d[0])
    mynode.append(d[1])
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID2=list(node_ID1)
#print(node_ID)
n=len(node_ID2)
print(n)
from natsort import natsorted, ns
node_ID=natsorted(node_ID2)
#print(node_ID)
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(d[0])][node_ID.index(d[1])]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

3312
 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
D_only=[]
S_only=[]
Both=[]

In [14]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

for it in range(10):
    p=[]
    q=[]
    p=random.sample(range(3312), 1000)
    #print(p)
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    #LABLE THE UNKNOWN/TEST NODE AS 6 AND PLACE CLASS TO THE DIAGONAL OF ADJACENCY MATRIX

    class_60=np.array(catagories)
    for d in p:
        class_60[d]=6
    #print(class_60)
    for i in range(n):
        A[i][i]=class_60[i]

    Node_class=[0,1,2,3,4,5]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['subject'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

 
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)

    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=len(F_vec[0])
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    for i in range(6):
        feature.append("I_{}".format(i))
    for i in range(6):
        feature.append("S_{}".format(i))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    for i in range(6):
        feature.append("I_{}".format(i))
    for i in range(6):
        feature.append("S_{}".format(i))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

In [ ]:
print(Ac_domain2)

In [15]:

mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial Only
Mean: 66.71000000000001
Standard Deviation: 1.6120725100868678
Domain only
Mean: 89.51
Standard Deviation: 0.9279008567729642
Accuracy using both
Mean: 88.43
Standard Deviation: 1.0924488292109844
